In [1]:
from glob import glob
from pathlib import Path
from langsmith import Client

client = Client()

IMAGE_DIR = "./example_images"


# Get all the images in the directory and extract the file name
def get_image_files(directory=IMAGE_DIR):
    return [
        {
            "image_path": image,
            "messages": [],
            "chat_history": [],
            "actual_label": Path(image).stem.split("_")[0],
        }
        for image in glob(f"{directory}/*")
    ]

In [2]:
from farmGPT.agent_workflow import compile_graph
from langchain_core.runnables import RunnableLambda
from models import load_llm
from dotenv import load_dotenv

load_dotenv(override=True)
import re


def extract_name(text):
    for category in ["Pest", "Disease"]:
        pattern = rf"\*\*{category}\*\*: (.*?)\n\n"
        match = re.search(pattern, text)
        if match:
            return match.group(1)
    return None


model = load_llm(model="gpt-3.5-turbo-0125")
chain = compile_graph(llm=model) | (lambda x: x.content) | RunnableLambda(extract_name)

In [3]:
chain.get_graph().draw_mermaid_png(output_file_path="./graph.png")

b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xe2\x01\xd8ICC_PROFILE\x00\x01\x01\x00\x00\x01\xc8\x00\x00\x00\x00\x040\x00\x00mntrRGB XYZ \x07\xe0\x00\x01\x00\x01\x00\x00\x00\x00\x00\x00acsp\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\xf6\xd6\x00\x01\x00\x00\x00\x00\xd3-\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\tdesc\x00\x00\x00\xf0\x00\x00\x00$rXYZ\x00\x00\x01\x14\x00\x00\x00\x14gXYZ\x00\x00\x01(\x00\x00\x00\x14bXYZ\x00\x00\x01<\x00\x00\x00\x14wtpt\x00\x00\x01P\x00\x00\x00\x14rTRC\x00\x00\x01d\x00\x00\x00(gTRC\x00\x00\x01d\x00\x00\x00(bTRC\x00\x00\x01d\x00\x00\x00(cprt\x00\x00\x01\x8c\x00\x00\x00<mluc\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x0cenUS\x00\x00\x00\x08\x00\x00\x00\x1c\x00s\x00R\x00G\x00BXYZ \x00\x00\x00\x00

In [4]:
template = """
You are an agricultural expert tasked with evaluating the performance of an AI model designed to identify diseases, pests, and insects from images. 
The model will provide a prediction, and you will compare it with the actual label.

When evaluating the predictions, you should:

1. Disregard any spelling mistakes or minor variations in the predicted and actual labels.
2. Consider alternative names or synonyms for the diseases, pests, and insects.

For example, if the predicted label is "Corn Earworm" and the actual label is "Earworm on Corn," you should respond with "Y" since they refer to the same thing, despite the slight variation in wording.

Prediction: {prediction}

Actual Label: {actual_label}
"""

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq.chat_models import ChatGroq
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Literal
from langchain_core.runnables import RunnablePassthrough


class Evaluation(BaseModel):
    """Use this tool to evaluate the performance of an AI model designed to identify diseases, pests, and insects from images."""
    evaluation: Literal["Y", "N"] = Field(
        ...,
        description="Evaluation of the prediction. Reply with 'Y' if the prediction is accurate, and 'N' if it is inaccurate.",
    )
    prediction: str = Field(..., description="The predicted label.")
    actual_label: str = Field(..., description="The actual label.")


prompt = ChatPromptTemplate.from_template(template)
eval_chain = prompt | ChatGroq(
    model="Llama3-70b-8192", temperature=0
).with_structured_output(Evaluation)

c:\Users\PC\Cropify\.venv\lib\site-packages\langchain_core\_api\beta_decorator.py:87: LangChainBetaWarning: The method `ChatGroq.with_structured_output` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


In [6]:
import time
import asyncio

full_chain = {"prediction": chain, "actual_label": RunnablePassthrough()} | eval_chain

score = []

image_files = get_image_files()
batch_size = 2
sleep_time = 80

for i in range(0, len(image_files), batch_size):
    batch = image_files[i:i+batch_size]
    output = await full_chain.abatch(batch, config={"max_concurrency": 2})
    for item in output:
        score.append(item.dict())
    if i + batch_size < len(image_files):
        time.sleep(sleep_time)

label='crop_disease'
label='crop_disease'
label='crop_disease'
label='crop_disease'
label='crop_disease'
label='pest'
label='pest'
label='crop_disease'
label='crop_disease'
label='pest'
label='pest'
label='crop_disease'
label='crop_disease'
label='crop_disease'
label='crop_disease'
label='pest'
label='crop_disease'
label='crop_disease'
label='crop_disease'
label='crop_disease'
label='pest'
label='crop_disease'


In [9]:
import json


with open("./score.json", "w") as f:
    json.dump(score, f, indent=4)

0.25